<a href="https://colab.research.google.com/github/YogeshParihar1311/Deep_Learning/blob/main/sms_spam_with_ML_DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats
from statsmodels.stats.outliers_influence import variance_inflation_factor
# from autoviz.classify_method import data_cleaning_suggestions ,data_suggestions

# from pycaret  import regression
from sklearn.model_selection import cross_val_score

In [2]:
from google.colab import files

files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"yogeshparihar1311","key":"41646bae978e8cfc336f2cbe53f584d0"}'}

In [3]:
! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/

In [4]:
 ! chmod 600 ~/.kaggle/kaggle.json

In [5]:
! kaggle datasets download uciml/sms-spam-collection-dataset

100% 211k/211k [00:00<00:00, 609kB/s]
100% 211k/211k [00:00<00:00, 609kB/s]


In [6]:
! unzip sms-spam-collection-dataset.zip

Archive:  sms-spam-collection-dataset.zip
  inflating: spam.csv                


In [7]:
df = pd.read_csv("spam.csv",encoding='cp1252')

In [8]:
df.shape

(5572, 5)

In [9]:
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   v1          5572 non-null   object
 1   v2          5572 non-null   object
 2   Unnamed: 2  50 non-null     object
 3   Unnamed: 3  12 non-null     object
 4   Unnamed: 4  6 non-null      object
dtypes: object(5)
memory usage: 217.8+ KB


In [11]:
df.describe()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
count,5572,5572,50,12,6
unique,2,5169,43,10,5
top,ham,"Sorry, I'll call later","bt not his girlfrnd... G o o d n i g h t . . .@""","MK17 92H. 450Ppw 16""","GNT:-)"""
freq,4825,30,3,2,2


In [12]:
df.isnull().sum()/df.shape[0]*100

v1             0.000000
v2             0.000000
Unnamed: 2    99.102656
Unnamed: 3    99.784637
Unnamed: 4    99.892319
dtype: float64

In [13]:
df = df[['v1','v2']]

In [14]:
df.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [15]:
df.columns = ['target','text']

In [16]:
df.head()

,target,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [17]:
df.duplicated().sum()

403

In [18]:
df.drop_duplicates(inplace=True)

In [19]:
# removing html tags
import re
def remove_tags(text):
    cleaned_txt = re.sub(re.compile('<.*?>'),'',text)
    return cleaned_txt

In [20]:
# removing urls
def remove_urls(text):
    pattern = re.sub(re.compile(r'https?://\S+|www.\S+'),'',text)
    return pattern

In [21]:
# remove punctuation
import string
exclude = string.punctuation
def remove_punc(text):
    return text.translate(str.maketrans('','',exclude))

In [22]:
# remove chat words
chat_words = {
    "AFAIK":"As Far As I Know",
"AFK":"Away From Keyboard",
"ASAP":"As Soon As Possible",
"ATK":"At The Keyboard",
"ATM":"At The Moment",
"A3":"Anytime, Anywhere, Anyplace",
"BAK":"Back At Keyboard",
"BBL":"Be Back Later",
"BBS":"Be Back Soon",
"BFN":"Bye For Now",
"B4N":"Bye For Now",
"BRB":"Be Right Back",
"BRT":"Be Right There",
"BTW":"By The Way",
"B4":"Before",
"B4N":"Bye For Now",
"CU":"See You",
"CUL8R":"See You Later",
"CYA":"See You",
"FAQ":"Frequently Asked Questions",
"FC":"Fingers Crossed",
"FWIW":"For What It's Worth",
"FYI":"For Your Information",
"GAL":"Get A Life",
"GG":"Good Game",
"GN":"Good Night",
"GMTA":"Great Minds Think Alike",
"GR8":"Great!",
"G9":"Genius",
"IC":"I See",
"ICQ":"I Seek you",
"ILU":"I Love You",
"IMHO":"In My Honest/Humble Opinion",
"IMO":"In My Opinion",
"IOW":"In Other Words",
"IRL":"In Real Life",
"KISS":"Keep It Simple, Stupid",
"LDR":"Long Distance Relationship",
"LMAO":"Laugh My A.. Off",
"LOL":"Laughing Out Loud",
"LTNS":"Long Time No See",
"L8R":"Later",
"MTE":"My Thoughts Exactly",
"M8":"Mate",
"NRN":"No Reply Necessary",
"OIC":"Oh I See",
"PITA":"Pain In The A..",
"PRT":"Party",
"PRW":"Parents Are Watching",
"QPSA":"Que Pasa",
"ROFL":"Rolling On The Floor Laughing",
"ROFLOL":"Rolling On The Floor Laughing Out Loud",
"ROTFLMAO":"Rolling On The Floor Laughing My A.. Off",
"SK8":"Skate",
"STATS":"Your sex and age",
"ASL":"Age, Sex, Location",
"THX":"Thank You",
"TTFN":"Ta-Ta For Now!",
"TTYL":"Talk To You Later",
"U":"You",
"U2":"You Too",
"U4E":"Yours For Ever",
"WB":"Welcome Back",
"WTF":"What The F...",
"WTG":"Way To Go!",
"WUF":"Where Are You From?",
"W8":"Wait...",
"7K":"Sick:-D Laugher",
}

In [23]:
chat_words = []
def chat_conversion(text):
    new_text = []
    for w in text.split():
        if w.upper() in chat_words:
            new_text.append(chat_words[w.upper()])
        else:
            new_text.append(w)
    return " ".join(new_text)

In [24]:
# spelling correction
from textblob import TextBlob
def spell_correction(text):
    textblob = TextBlob(text)
    return textblob.correct().string

In [25]:
! pip install emoji

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.5/397.5 kB 6.5 MB/s eta 0:00:00


In [26]:
# replace emoji
import emoji
def replace_emoji(text):
    return emoji.demojize(text)

In [27]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [28]:
from nltk.corpus import stopwords
stopwords.words('english')

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [29]:
# remove stop words
from nltk.corpus import stopwords
def remove_stopwords(text):
    new_text = []
    for word in text.split():
        if word in stopwords.words('english'):
            new_text.append('')
        else:
            new_text.append(word)
    x = new_text[:]
    new_text.clear()
    return ' '.join(x)

In [30]:
def transform_text(text):
    txt = remove_tags(text)
    txt1 = remove_urls(txt)
    txt2 = remove_punc(txt1)
    txt3 = chat_conversion(txt2)
    # txt4 = spell_correction(txt3)
    txt5 = replace_emoji(txt3)
    txt6 = remove_stopwords(txt5)
    return txt6.lower()

In [31]:
df['text'][453]

'Ok i will tell her to stay out. Yeah its been tough but we are optimistic things will improve this month.'

In [32]:
df['transformed_text'] = df['text'].apply(transform_text)

In [33]:
df.head()

,target,text,transformed_text
0,ham,"Go until jurong point, crazy.. Available only ...",go jurong point crazy available bugis n gre...
1,ham,Ok lar... Joking wif u oni...,ok lar joking wif u oni
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entry 2 wkly comp win fa cup final tkt...
3,ham,U dun say so early hor... U c already then say...,u dun say early hor u c already say
4,ham,"Nah I don't think he goes to usf, he lives aro...",nah i dont think goes usf lives around though


In [34]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['target'] = le.fit_transform(df['target'])

In [35]:
df.head()

,target,text,transformed_text
0,0,"Go until jurong point, crazy.. Available only ...",go jurong point crazy available bugis n gre...
1,0,Ok lar... Joking wif u oni...,ok lar joking wif u oni
2,1,Free entry in 2 a wkly comp to win FA Cup fina...,free entry 2 wkly comp win fa cup final tkt...
3,0,U dun say so early hor... U c already then say...,u dun say early hor u c already say
4,0,"Nah I don't think he goes to usf, he lives aro...",nah i dont think goes usf lives around though


In [36]:
df['target'].value_counts()

0    4516
1     653
Name: target, dtype: int64

In [ ]:
X = df['transformed_text']
y = df['target']

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [ ]:
X_train_tfidf = tfidf.fit_transform(X_train).toarray()
X_test_tfidf = tfidf.transform(X_test).toarray()

In [ ]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()

In [ ]:
nb.fit(X_train_tfidf,y_train)

MultinomialNB()

In [ ]:
y_pred = nb.predict(X_test_tfidf)

In [ ]:
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report

In [ ]:
accuracy_score(y_test,y_pred)

0.9613152804642167

In [ ]:
confusion_matrix(y_test,y_pred)

array([[889,   0],
       [ 40, 105]])

In [ ]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.96      1.00      0.98       889
           1       1.00      0.72      0.84       145

    accuracy                           0.96      1034
   macro avg       0.98      0.86      0.91      1034
weighted avg       0.96      0.96      0.96      1034



In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()

In [ ]:
rfc.fit(X_train_tfidf,y_train)
y_pred = rfc.predict(X_test_tfidf)
print(accuracy_score(y_test,y_pred))
print('*'*80)
print(confusion_matrix(y_test,y_pred))
print('*'*80)
print(classification_report(y_test,y_pred))

0.9671179883945842
********************************************************************************
[[888   1]
 [ 33 112]]
********************************************************************************
              precision    recall  f1-score   support

           0       0.96      1.00      0.98       889
           1       0.99      0.77      0.87       145

    accuracy                           0.97      1034
   macro avg       0.98      0.89      0.92      1034
weighted avg       0.97      0.97      0.97      1034



In [ ]:
from xgboost import XGBClassifier
xgbc = XGBClassifier()

In [ ]:
xgbc.fit(X_train_tfidf,y_train)
y_pred = xgbc.predict(X_test_tfidf)
print(accuracy_score(y_test,y_pred))
print('*'*80)
print(confusion_matrix(y_test,y_pred))
print('*'*80)
print(classification_report(y_test,y_pred))

0.9613152804642167
********************************************************************************
[[882   7]
 [ 33 112]]
********************************************************************************
              precision    recall  f1-score   support

           0       0.96      0.99      0.98       889
           1       0.94      0.77      0.85       145

    accuracy                           0.96      1034
   macro avg       0.95      0.88      0.91      1034
weighted avg       0.96      0.96      0.96      1034



In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
etc = ExtraTreesClassifier()

In [ ]:
etc.fit(X_train_tfidf,y_train)
y_pred = etc.predict(X_test_tfidf)
print(accuracy_score(y_test,y_pred))
print('*'*80)
print(confusion_matrix(y_test,y_pred))
print('*'*80)
print(classification_report(y_test,y_pred))

0.9690522243713733
********************************************************************************
[[888   1]
 [ 31 114]]
********************************************************************************
              precision    recall  f1-score   support

           0       0.97      1.00      0.98       889
           1       0.99      0.79      0.88       145

    accuracy                           0.97      1034
   macro avg       0.98      0.89      0.93      1034
weighted avg       0.97      0.97      0.97      1034



In [ ]:
df.head()

,target,text,transformed_text
0,0,"Go until jurong point, crazy.. Available only ...",go jurong point crazy available bugis n gre...
1,0,Ok lar... Joking wif u oni...,ok lar joking wif u oni
2,1,Free entry in 2 a wkly comp to win FA Cup fina...,free entry 2 wkly comp win fa cup final tkt...
3,0,U dun say so early hor... U c already then say...,u dun say early hor u c already say
4,0,"Nah I don't think he goes to usf, he lives aro...",nah i dont think goes usf lives around though


In [ ]:
max_text_len = len(max(df["transformed_text"],key=len))
max_text_len

615

In [37]:
df.head()

,target,text,transformed_text
0,0,"Go until jurong point, crazy.. Available only ...",go jurong point crazy available bugis n gre...
1,0,Ok lar... Joking wif u oni...,ok lar joking wif u oni
2,1,Free entry in 2 a wkly comp to win FA Cup fina...,free entry 2 wkly comp win fa cup final tkt...
3,0,U dun say so early hor... U c already then say...,u dun say early hor u c already say
4,0,"Nah I don't think he goes to usf, he lives aro...",nah i dont think goes usf lives around though


In [42]:
corpus = list(df["transformed_text"])

In [47]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot

In [49]:
voc_size = 10000
one_hot_repr = [one_hot(words,voc_size) for words in corpus]

In [51]:
sent_length = 620
embedded_docs = pad_sequences(one_hot_repr,padding="pre",maxlen=sent_length)

In [54]:
len(embedded_docs),len(df)

(5169, 5169)

In [115]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Dropout,Embedding,LSTM

In [116]:
emb_vec_features = 50
model = Sequential()
model.add(Embedding(voc_size,emb_vec_features,input_length=sent_length))
model.add(LSTM(100))
# model.add(Dropout(0,4))
model.add(Dense(1,activation="sigmoid"))

In [117]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 620, 50)           500000    
                                                                 
 lstm (LSTM)                 (None, 100)               60400     
                                                                 
 dense_2 (Dense)             (None, 1)                 101       
                                                                 
Total params: 560501 (2.14 MB)
Trainable params: 560501 (2.14 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [118]:
model.compile(loss="binary_crossentropy",optimizer="Adam",metrics=["accuracy"])

In [123]:
X = np.array(embedded_docs)
y = np.array(df["target"])

In [124]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [125]:
model.fit(X_train,y_train,epochs=1,validation_data=(X_test,y_test),batch_size=64)

65/65 [==============================] - 10s 131ms/step - loss: 0.0606 - accuracy: 0.9839 - val_loss: 0.0666 - val_accuracy: 0.9816


In [128]:
y_score = model.predict(X_test)

33/33 [==============================] - 1s 15ms/step


In [129]:
y_pred = np.argmax(y_score,axis=1)

In [130]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.8597678916827853

In [160]:
emb_vec_features = 50
model = Sequential()
model.add(Embedding(voc_size,emb_vec_features,input_length=sent_length))
model.add(Dropout(0.4))
model.add(LSTM(100))
model.add(Dropout(0.4))
model.add(Dense(1,activation="sigmoid"))

In [161]:
model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_8 (Embedding)     (None, 620, 50)           500000    
                                                                 
 dropout_7 (Dropout)         (None, 620, 50)           0         
                                                                 
 lstm_9 (LSTM)               (None, 100)               60400     
                                                                 
 dropout_8 (Dropout)         (None, 100)               0         
                                                                 
 dense_7 (Dense)             (None, 1)                 101       
                                                                 
Total params: 560501 (2.14 MB)
Trainable params: 560501 (2.14 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [162]:
model.compile(loss="binary_crossentropy",optimizer="Adam",metrics=["accuracy"])

In [171]:
model.fit(X_train,y_train,epochs=10,validation_data=(X_test,y_test),batch_size=32)

Epoch 1/10
130/130 [==============================] - 4s 30ms/step - loss: 3.5433e-05 - accuracy: 1.0000 - val_loss: 0.1718 - val_accuracy: 0.9768
Epoch 2/10
130/130 [==============================] - 3s 26ms/step - loss: 3.3273e-05 - accuracy: 1.0000 - val_loss: 0.1663 - val_accuracy: 0.9778
Epoch 3/10
130/130 [==============================] - 5s 38ms/step - loss: 0.0080 - accuracy: 0.9969 - val_loss: 0.1005 - val_accuracy: 0.9671
Epoch 4/10
130/130 [==============================] - 4s 29ms/step - loss: 0.0014 - accuracy: 1.0000 - val_loss: 0.1313 - val_accuracy: 0.9778
Epoch 5/10
130/130 [==============================] - 3s 27ms/step - loss: 1.8526e-04 - accuracy: 1.0000 - val_loss: 0.1359 - val_accuracy: 0.9778
Epoch 6/10
130/130 [==============================] - 4s 28ms/step - loss: 1.0200e-04 - accuracy: 1.0000 - val_loss: 0.1417 - val_accuracy: 0.9787
Epoch 7/10
130/130 [==============================] - 4s 30ms/step - loss: 7.5274e-05 - accuracy: 1.0000 - val_loss: 0.1416 - 

In [168]:
y_score = model.predict(X_test)

33/33 [==============================] - 0s 13ms/step


In [169]:
y_pred = np.argmax(y_score,axis=1)

In [170]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.8597678916827853